In [ ]:
# LLM Models
from langchain_openai import OpenAI, ChatOpenAI
from langchain_ollama import OllamaLLM
from langchain_ollama import ChatOllama
import ollama

# from ollama import chat
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain.schema import SystemMessage, HumanMessage, AIMessage

# OutputParsers
from langchain.schema.output_parser import StrOutputParser

# Gradio frontend
import gradio as gr

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
book_filepath = {"A Christmas Carol": '.\\data\\A_Christmas_Carol.txt',
                 "A Farewell to Arms": '.\\data\\A_Farewell_to_Arms.txt',
                 "Adventures of Huckleberry Finn": '.\\data\\Adventures_of_Huckleberry_Finn.txt',
                 "Crime and Punishment": '.\\data\\Crime_and_Punishment.txt',
                 "Moby Dick": '.\\data\\Moby_Dick.txt', 
                 "Macbeth": '.\\data\\Macbeth.txt',
                 "Silas Marner": '.\\data\\Silas_Marner.txt'}

In [ ]:
def book_chapter_summary(llm='gpt-4o-mini', book_title="Moby Dick", chapter_num=1):
    
    filepath = book_filepath[book_title]
    
    with open(file=filepath, mode='r') as file:
        f = file.read()

    f_lines = f.split('\n')
    title = f_lines[0]
    author = f_lines[2].split('By ')[-1]
    chapters = f.split('[divider]')[1:]

    num_chapters = len(chapters)
    
    if chapter_num > num_chapters:
        chapter_selected = num_chapters
    else:
        chapter_selected = chapter_num
        
    chapter = chapters[chapter_selected - 1]
    
    
    prompt = ChatPromptTemplate.from_template("""
    Provide a summary of the following text, at most 200 words. Answer must be based only on the text given. 
    context: {chapter}
    """)

    if llm == 'gpt-4o-mini':
        chat_model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                                max_completion_tokens=256,
                                temperature=0.7)
    elif llm == 'llama3.2':
        chat_model = OllamaLLM(model="llama3.2",
                               num_ctx = 8192, 
                               num_predict = 256,
                               temperature = 0.7)

    chat_chain = prompt | chat_model | StrOutputParser()

    response = chat_chain.invoke({"chapter": chapters[chapter_num - 1]})

    return response

In [ ]:
interface = gr.Interface(fn = book_chapter_summary,
                         inputs=[gr.Dropdown(choices=['gpt-4o-mini', 'llama3.2'],
                                             value='gpt-4o-mini',
                                             multiselect=False,
                                             label="LLM Model"),
                                 gr.Dropdown(choices=['A Christmas Carol',
                                                      'A Farewell to Arms',
                                                      'Adventures of Huckleberry Finn',
                                                      'Crime and Punishment',
                                                      'Moby Dick',
                                                      'Macbeth',
                                                      'Silas Marner'],
                                             value='Moby Dick',
                                             multiselect=False,
                                     label="Title of the Book"), 
                                 gr.Number(label="Chapter Number", 
                                           value=1,
                                           minimum=1)],
                         outputs=[gr.Text(label="Summary")],
                         flagging_mode="never"
                        )

In [ ]:
interface.launch(share=False)

In [ ]:
interface.close()